In [2]:
# import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

2023-04-27 11:03:10.884415: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load dataset 
data = pd.read_csv('/Users/nadinejackson/Downloads/corruption-perceptions-index-cpi.csv')

# Convert data into DataFrame
data = pd.DataFrame([
    ['Benin', 42, 41, 41, 40, 39, 36, 37, 39, 36, 36],
    ['Botswana', 55, 60, 61, 61, 61, 60, 63, 63, 64, 65],
    ['Cabo Verde', 58, 58, 58, 57, 55, 59, 55, 57, 58, 60],
    ['Ethiopia', 39, 38, 37, 34, 35, 34, 33, 33, 33, 33],
    ['Kenya', 30, 31, 28, 27, 28, 26, 25, 25, 27, 27],
    ['Nigeria', 24, 25, 26, 27, 27, 28, 26, 27, 25, 27],
    ['Senegal', 43, 45, 45, 45, 45, 45, 44, 43, 41, 36],
    ['South Africa', 44, 44, 44, 43, 43, 45, 44, 44, 42, 43],
    ['Sudan', 20, 16, 16, 16, 16, 14, 12, 11, 11, 13],
    ['Tanzania', 39, 38, 37, 36, 36, 32, 30, 31, 33, 35],
    ['Zambia', 33, 33, 34, 35, 37, 38, 38, 38, 38, 37]
], columns=['Country', 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012])


In [4]:
# Preprocess the data
text_data = ''
for _, row in data.iterrows():
    country = row['Country']
    for year in range(2012, 2022):
        text_data += f"{country} has a corruption score of {row[year]} in {year}. "

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([text_data])

sequence_data = tokenizer.texts_to_sequences([text_data])[0]
vocab_size = len(tokenizer.word_index) + 1

seq_length = 100
input_sequences = []
output_char = []

for i in range(0, len(sequence_data) - seq_length):
    input_sequences.append(sequence_data[i:i+seq_length])
    output_char.append(sequence_data[i+seq_length])

X = np.array(input_sequences)
y = tf.keras.utils.to_categorical(output_char, num_classes=vocab_size)


In [5]:
# Tokenize and pad the sequences
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts([text_data])

sequence_data = tokenizer.texts_to_sequences([text_data])[0]
vocab_size = len(tokenizer.word_index) + 1

seq_length = 100
input_sequences = []
output_char = []

for i in range(0, len(sequence_data) - seq_length):
    input_sequences.append(sequence_data[i:i+seq_length])
    output_char.append(sequence_data[i+seq_length])

X = np.array(input_sequences)
y = tf.keras.utils.to_categorical(output_char, num_classes=vocab_size)

In [6]:
# Define the RNN model using LSTM layers
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=seq_length))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

2023-04-27 11:03:17.481104: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-27 11:03:17.485507: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-27 11:03:17.487630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          4608      
                                                                 
 lstm (LSTM)                 (None, 100, 256)          394240    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 36)                9252      
                                                                 
Total params: 933,412
Trainable params: 933,412
Non-trainable params: 0
_________________________________________________________________


2023-04-27 11:03:17.874811: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-27 11:03:17.878206: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-27 11:03:17.881464: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [7]:
# Split the data into training and validation sets
split_ratio = 0.8
train_size = int(len(X) * split_ratio)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

# Train the model with early stopping based on validation loss
checkpoint = ModelCheckpoint("corruption_rnn.h5", monitor='val_loss', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=64, callbacks=[checkpoint, early_stopping])

Epoch 1/100


2023-04-27 11:03:19.007728: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-27 11:03:19.016705: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-27 11:03:19.033621: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

63/63 [==============================] - ETA: 0s - loss: 3.0043 - accuracy: 0.1877

2023-04-27 11:04:47.763660: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-27 11:04:47.766651: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-27 11:04:47.769225: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_loss improved from inf to 2.87730, saving model to corruption_rnn.h5
63/63 [==============================] - 97s 1s/step - loss: 3.0043 - accuracy: 0.1877 - val_loss: 2.8773 - val_accuracy: 0.1970
Epoch 2/100
63/63 [==============================] - ETA: 0s - loss: 2.3422 - accuracy: 0.2828
Epoch 2: val_loss improved from 2.87730 to 1.91022, saving model to corruption_rnn.h5
63/63 [==============================] - 107s 2s/step - loss: 2.3422 - accuracy: 0.2828 - val_loss: 1.9102 - val_accuracy: 0.4110
Epoch 3/100
63/63 [==============================] - ETA: 0s - loss: 1.5037 - accuracy: 0.5332
Epoch 3: val_loss improved from 1.91022 to 1.27663, saving model to corruption_rnn.h5
63/63 [==============================] - 109s 2s/step - loss: 1.5037 - accuracy: 0.5332 - val_loss: 1.2766 - val_accuracy: 0.6350
Epoch 4/100
63/63 [==============================] - ETA: 0s - loss: 0.9862 - accuracy: 0.7092
Epoch 4: val_loss improved from 1.27663 to 1.00528, saving model to cor

In [8]:
# Generate text using the trained model
def generate_text(seed_text, model, tokenizer, seq_length, num_chars):
    output_text = seed_text

    for _ in range(num_chars):
        tokenized_text = tokenizer.texts_to_sequences([seed_text])[-1]
        padded_text = pad_sequences([tokenized_text], maxlen=seq_length)

        probabilities = model.predict(padded_text)[0]
        predicted_index = np.argmax(probabilities)
        predicted_char = tokenizer.index_word[predicted_index]

        output_text += predicted_char
        seed_text += predicted_char

    return output_text

In [10]:
seed_text = "Nigeria has a corruption score of "
generated_text = generate_text(seed_text, model, tokenizer, seq_length, 11)
print(generated_text)


1/1 [==============================] - 0s 107ms/step
Nigeria has a corruption score of 44 in 2018.
